In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# import matplotlib.pyplot as plt
# from scipy.spatial.distance import pdist, squareform

In [ ]:
yelp_df = pd.read_csv('veg.csv', sep=';', header=None, names = ["rev_id", "stars", "date", "text", "useful", "funny", "cool", "name", "city", "state", "category"])

In [2]:
vectorizer = TfidfVectorizer(stop_words='english')

In [4]:
X = vectorizer.fit_transform(yelp_df['text'])

In [6]:
y = yelp_df['category']